Building the Trading System

Using Decimal to represent money

In [ ]:
val = 0.1
total = 0.0
for i in range(10):
    total += val
total

In [ ]:
from decimal import Decimal
dp = Decimal('0.2')
val = Decimal(0.1).quantize(dp)
total = Decimal(0.0).quantize(dp)
for i in range(10):
    total += val
total

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Order:
    sym: str
    signed_qty: Decimal

    def __str__(self) -> str:
        sign = "LONG" if self.signed_qty > 0 else "SHORT"
        return f"Order({sign} {self.signed_qty} {self.sym})"

In [ ]:
from decimal import Decimal

def decimal_sign(d: Decimal) -> int:
    return 1 if d > Decimal(0) else -1

def is_long(x: Decimal) -> bool:
    return decimal_sign(x) > 0

@dataclass(frozen=True)
class Trade:
    sym: str
    signed_qty: Decimal
    price: Decimal
    pnl: Decimal

    def __str__(self) -> str:
        sign = "LONG" if is_long(self.signed_qty) else "SHORT"
        return f"Trade({sign} {self.signed_qty} {self.sym} {self.price} {self.pnl})"

    def is_long(self) -> bool:
        return is_long(self.signed_qty)

In [ ]:
@dataclass
class Position:
    sym: str
    signed_qty: Decimal
    price: Decimal

    def close(self) -> "Order":
        return Order(self.sym, -self.signed_qty)

    def is_long(self) -> bool:
        return is_long(self.signed_qty)

    def unrealized_pnl(self, current_price: Decimal) -> Decimal:
        entry_val = self.price * self.signed_qty
        exit_val = current_price * -self.signed_qty
        return entry_val + exit_val

In [ ]:
from abc import ABC, abstractmethod
from decimal import Decimal

class Account(ABC):
    @abstractmethod
    def balance(self) -> Decimal:
        pass

    @abstractmethod
    def get_position(self, sym: str) -> Optional[Position]:
        pass

In [ ]:
from decimal import Decimal
from typing import Dict, List, Optional

class TestAccount(Account):
    """A simulated account for testing or paper trading."""

    def __init__(self, _balance: Decimal) -> None:
        self._balance = _balance
        self._positions: Dict[str, Position] = {}
        self._trades: List[Trade] = []

    def balance(self) -> Decimal:
        return self._balance

    def get_position(self, sym) -> Optional[Position]:
        return self._positions.get(sym)

    def __repr__(self) -> str:
        return f"TestAccount(balance={self._balance}, positions={self._positions}, trades={self._trades})"

In [ ]:
acc = TestAccount(Decimal(50.0))
acc.balance()

In [ ]:
acc

Model an Exchange

In [ ]:
from abc import abstractmethod
from decimal import Decimal

class Exchange(Account):
    """Abstract base class representing a trading exchange/broker."""

    @abstractmethod
    def market_order(self, sym: str, signed_qty: Decimal, price: Decimal) -> Trade:
        """Execute a market order and return a Trade result."""
        pass

    @abstractmethod
    def limit_order(self, sym: str, signed_qty: Decimal, price: Decimal, post_only: bool = False) -> Optional[Trade]:
        """Execute a limit order and return a Trade if it crosses book."""
        pass

In [ ]:
from typing import Dict,List

class TestExchange(Exchange):
    _account: TestAccount

    def __init__(self, account: TestAccount):
        self._account = account

    def market_order(self, sym: str, signed_qty: Decimal, price: Decimal) -> "Trade":
        # Update balance and position
        trade = self._update_position(sym, signed_qty, price)
        self._account._balance += trade.pnl
        self._account._trades.append(trade)
        return trade

    def _update_position(self, sym: str, signed_qty, price: Decimal) -> Trade:
        position = self._account._positions.pop(sym, None)
        pnl = Decimal(0.0)
        if position is not None:
            entry_val = position.price * position.signed_qty
            exit_val = price * position.signed_qty
            pnl = exit_val - entry_val
        else:
            self._account._positions[sym] = Position(sym, signed_qty, price)
        return Trade(sym, signed_qty, price, pnl)

    def limit_order(self, sym, signed_qty, price, post_only = False):
        raise Exception("not yet implemented")

    def balance(self) -> Decimal:
        return self._account.balance()

    def get_position(self, sym) -> Optional[Position]:
        return self._account.get_positions(sym)

    def __repr__(self) -> str:
        return f"TestExchange(balance={self.balance()}, positions={self._account._positions}, trades={self._account._trades})"

Open Position

In [ ]:
exchange = TestExchange(TestAccount(Decimal(50.0)))

price = Decimal(10)
qty = Decimal(5.0)
exchange.market_order('BTCUSDT', qty, Decimal(price))

In [ ]:
exchange = TestExchange(TestAccount(Decimal(50.0)))

price = Decimal(10)
qty = Decimal(5.0)
exchange.market_order('BTCUSDT', qty, Decimal(price))

Close Position

In [ ]:
price = Decimal(15.0)
exchange.market_order('BTCUSDT', -qty, price)

In [ ]:
exchange

In [ ]:
entry_notional_value = Decimal(5) * Decimal(10)
entry_notional_value

In [ ]:
exit_notional_val = Decimal(5) * Decimal(15)
exit_notional_val

In [ ]:
exit_notional_val - entry_notional_value

Building Strategy API

In [ ]:
class Strategy(ABC):
    @abstractmethod
    def on_tick(self, price: float, account: Account) -> Optional[List[Order]]:
        pass

Implementning our strategy

In [ ]:
import torch.nn as nn

class BasicTakerStrat(Strategy):
    def __init__(self,
                 sym: str,
                 model: nn.Module,
                 log_return_lags: LogReturnLags,
                 scale_factor: Decimal = None) -> None:
        self.sym = sym
        self.model = model
        self.log_return_lags = log_return_lags
        if scale_factor is None:
            scale_factor = Decimal(1.0)
        self.scale_factor = Decimal(scale_factor)

    def _signed_compound_trade_size(self, y_hat: float, account: Account, cur_price: Decimal, position: Optional[Position]) -> Decimal:
        dir_signal = np.sign(y_hat)
        cur_balance =  account.balance()
        unrealized_balance = cur_balance + (position.unrealized_pnl(cur_price) if position else Decimal(0.0))
        qty = unrealized_balance / cur_price
        signed_qty = Decimal(dir_signal) * qty
        return signed_qty * self.scale_factor

    def _create_orders(self, y_hat: torch.Tensor, account: Account, price: Decimal) -> List[Order]:
        position = account.get_position(self.sym)
        signed_trade_size = self._signed_compound_trade_size(y_hat.item(), account, price, position)
        open_order = Order(self.sym, signed_trade_size)
        if position is not None:
            close_order = Order(position.sym, -position.signed_qty)
            return [close_order, open_order]
        return [open_order]

    def on_tick(self, price: float, account: Account) -> List[Order]:
        X = self.log_return_lags.on_tick(price)
        if X is not None:
            with torch.no_grad():
                y_hat = self.model(X)
                orders = self._create_orders(y_hat, account, Decimal(price))
                return orders
        return []

In [ ]:
# Window to stream lagged log returns
lags = LogReturnLags(3)
# Create Account
acc = TestAccount(Decimal(100.0))
# Create strategy
strat = BasicTakerStrat('BTCUSDT', model, lags, Decimal(1.0))

# First 12 hour interval - 2025/10/20 00:00
strat.on_tick(10.0, acc)

In [ ]:
# Second 12 hour interval - 2025/10/20 12:00
strat.on_tick(120.0, acc)

In [ ]:
# Third 12 hour interval - 2025/10/21 00:00
strat.on_tick(90.0, acc)

In [ ]:
# Fourth 12 hour interval - 2025/10/21 12:00
orders = strat.on_tick(100, acc)
orders

Execute Order

In [ ]:
exchange = TestExchange(acc)
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 100)

In [ ]:
exchange

In [ ]:
orders = strat.on_tick(115, acc)
orders

In [ ]:
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 115)

In [ ]:
exchange

In [ ]:
order = orders[1]
exchange.market_order(order.sym, order.signed_qty, 115)

In [ ]:
exchange

In [ ]:
orders = strat.on_tick(100, acc)
orders

In [ ]:
order = orders[0]
exchange.market_order(order.sym, order.signed_qty, 100)

In [ ]:
exchange

In [ ]:
order = orders[1]
exchange.market_order(order.sym, order.signed_qty, 100)

In [ ]:
exchange